In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

 **SETUP**

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

In [ ]:
#function for ordering cluster numbers
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [ ]:
# get a sample for quick analysis
df = pd.read_csv('/kaggle/input/sales-data-for-a-chain-of-brazilian-stores/Sales Report.csv', sep=';', parse_dates=['Sale Date Time'])


In [ ]:
df.sample(100)

In [ ]:
df['Date'] = pd.to_datetime(df['Sale Date Time'])



**create a generic user dataframe to keep Client and new segmentation scores**

In [ ]:
dfclient = pd.DataFrame(df['Client'].unique())
dfclient.columns = ['Client']


df_max_purchase = df.groupby('Client').Date.max().reset_index()
df_max_purchase.columns = ['Client','MaxPurchaseDate']
df_max_purchase['Recency'] = (df_max_purchase['MaxPurchaseDate'].max() - df_max_purchase['MaxPurchaseDate']).dt.days
dfclient = pd.merge(dfclient, df_max_purchase[['Client','Recency']], on='Client')

#build 4 clusters for recency and add it to dataframe
kmeans = KMeans(n_clusters=4)
kmeans.fit(dfclient[['Recency']])
dfclient['RecencyCluster'] = kmeans.predict(dfclient[['Recency']])   
dfclient = order_cluster('RecencyCluster', 'Recency',dfclient,False)

dfclient.head(100)

In [ ]:
#FREQUENCY
                
#get order counts for each user and create a dataframe with it
dffreq = df.groupby('Client').Date.count().reset_index()
dffreq.columns = ['Client','Frequency']
                
#add this data to our main dataframe
dfclient = pd.merge(dfclient, dffreq, on='Client')

#k-means
kmeans = KMeans(n_clusters=4)
kmeans.fit(dfclient[['Frequency']])
dfclient['FrequencyCluster'] = kmeans.predict(dfclient[['Frequency']])

#order the frequency cluster
dfclient = order_cluster('FrequencyCluster', 'Frequency',dfclient,True)

In [ ]:
#REVENUE
#calculate revenue for each customer
dfrev = df.groupby('Client').Total.sum().reset_index()

#merge it with our main dataframe
dfclient = pd.merge(dfclient, dfrev, on='Client')
#apply clustering
kmeans = KMeans(n_clusters=4)
kmeans.fit(dfclient[['Total']])
dfclient['TotalCluster'] = kmeans.predict(dfclient[['Total']])
#order the cluster numbers
dfclient = order_cluster('TotalCluster', 'Total',dfclient,True)

dfclient.head(100)

In [ ]:
#calculate overall score and use mean() to see details
dfclient['OverallScore'] = dfclient['RecencyCluster'] + dfclient['FrequencyCluster'] + dfclient['TotalCluster']
dfclient.groupby('OverallScore')['Recency','Frequency','Total'].mean()

dfclient['Segment'] = 'Valor Baixo'
dfclient.loc[dfclient['OverallScore']>2,'Segment'] = 'Valor Médio' 
dfclient.loc[dfclient['OverallScore']>4,'Segment'] = 'Valor Alto' 

dfclient['RecencyText'] = 'Inativo à muitíssimo tempo'
dfclient.loc[dfclient['RecencyCluster'] == 3, 'RecencyText'] = 'Ativo recentemente'
dfclient.loc[dfclient['RecencyCluster'] == 2, 'RecencyText'] = 'Inativo à pouco tempo'
dfclient.loc[dfclient['RecencyCluster'] == 1, 'RecencyText'] = 'Inativo à muito tempo'

dfclient['FrequencyText'] = 'Frequência Baixíssima'
dfclient.loc[dfclient['FrequencyCluster'] == 3, 'FrequencyText'] = 'Frequência Alta'
dfclient.loc[dfclient['FrequencyCluster'] == 2, 'FrequencyText'] = 'Frequência Média'
dfclient.loc[dfclient['FrequencyCluster'] == 1, 'FrequencyText'] = 'Frequência Baixa'

dfclient['TotalText'] = 'Receita Muito Baixa'
dfclient.loc[dfclient['TotalCluster'] == 3, 'TotalText'] = 'Receita Alta'
dfclient.loc[dfclient['TotalCluster'] == 2, 'TotalText'] = 'Receita Média'
dfclient.loc[dfclient['TotalCluster'] == 1, 'TotalText'] = 'Receita Baixa'


dfclient.head(100)